In [ ]:
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.models import load_model
from PIL import Image
import numpy as np
import flask
from flask import Flask, flash, request, redirect, url_for
import io
import cv2
import face_recognition as FR

In [ ]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None
UPLOAD_FOLDER = './upload'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [ ]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
def load_my_model():
    # load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    global model
    model = load_model('./model3.h5')
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

def prepare_image(image, target):
    # if the image mode is not RGB, convert it
    npImage = numpy.asarray(image)
    faces = FR.face_locations(npImage, number_of_times_to_upsample=0, model="hog")
    if len(faces) != 1 :
        print('fail to find : multi faces')
        return False
    T, R, B, L = faces[0]
    crop  = image[T:B, L:R]
    faceImage = cv2.resize(crop, (256, 256),interpolation = cv2.INTER_CUBIC)
    x = np.expand_dims(Image.img_to_array(faceImage), axis=0)
    image = np.vstack([x])
    # return the processed image
    return image

@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        print('post')
        if 'file' not in flask.request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        
        image = 0       
        if file and allowed_file(file.filename):
            extention = magic.from_file(file).split()[0].upper()
            if extention == 'GIF':
                imageObject = Image.open(predictPath+predictImage)
                imageObject.seek(0) 
                imageObject = imageObject.convert('RGB')
                image = np.array(imageObject)
            elif extention != 'JPEG' and extention != 'PNG':
                flash('fuck you')
            else:
                image = FR.load_image_file(predictPath+predictImage)       
        print(type(image))
        # preprocess the image and prepare it for classification
        image = prepare_image(image, target=(224, 224))

        # classify the input image and then initialize the list
        # of predictions to return to the client
        preds = model.predict(image)
        results = imagenet_utils.decode_predictions(preds)
        data["predictions"] = []

        # loop over the results and add them to the list of
        # returned predictions
        for (imagenetID, label, prob) in results[0]:
            r = {"label": label, "probability": float(prob)}
            data["predictions"].append(r)

        # indicate that the request was a success
        data["success"] = True

    # return the data dictionary as a JSON response
    return flask.jsonify(data)

In [ ]:
# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    application.secret_key = 'super secret key'
    application.config['SESSION_TYPE'] = 'filesystem'
    load_my_model()
    app.run()